In [1]:
%%capture
!pip install transformers==4.39.2

In [2]:
import os

os.environ['MODEL_NAME']='sentence-transformers/all-MiniLM-L6-v2'

In [3]:
from transformers import AutoTokenizer, AutoModel

tokenizer=AutoTokenizer.from_pretrained(os.getenv('MODEL_NAME'))

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
model=AutoModel.from_pretrained(os.getenv('MODEL_NAME')).to('cuda')

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

# One Sentence

In [5]:
text="Melbourne"
input_ids=tokenizer(text, return_tensors='pt',max_length=512, padding=True, truncation=True).to('cuda')

In [6]:
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

with torch.no_grad():
    model_output = model(**input_ids)
    
# Perform pooling
sentence_embeddings = mean_pooling(model_output, input_ids['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings.shape)


Sentence embeddings:
torch.Size([1, 384])


# Multiple Sentences

In [7]:
text_ls=['Melbourne', 'Sydney']
input_ids=tokenizer(text_ls, return_tensors='pt',max_length=512, padding=True, truncation=True).to('cuda')

with torch.no_grad():
    model_output = model(**input_ids)
    
# Perform pooling
sentence_embeddings = mean_pooling(model_output, input_ids['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings.shape)

Sentence embeddings:
torch.Size([2, 384])
